# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

## Import pandas

We are using the very handy pandas library for dataframes.

In [138]:
import bibtexparser

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [139]:
parser = bibtexparser.bparser.BibTexParser(common_strings=True, ignore_nonstandard_types=False)
with open("bibliography.bib") as bibtex_file:
    publications = bibtexparser.load(bibtex_file, parser=parser)

publications.entries

[{'file': 'PDF:/Users/tanjunkai/Zotero/storage/TWI9LZWW/Tan 等 - 2025 - Prescribed Performance Robust Approximate Optimal Tracking Control Via Stackelberg Game.pdf:application/pdf',
  'keywords': '/unread',
  'note': '{GSCC}: 0000000 2025-03-25T02:45:39.915Z \n{JCR分区}: Q1\n中科院分区升级版: 计算机科学2区\n影响因子: 5.9\n5年影响因子: 6.0\n{EI}: 是',
  'langid': 'english',
  'date': '2025-03-07',
  'urldate': '2025-03-18',
  'author': 'Tan, Junkai and Xue, Shuangsi and Li, Huan and Guo, Zihang and Cao, Hui and Li, Dongyu',
  'shortjournal': '{IEEE} Trans. Automat. Sci. Eng.',
  'journaltitle': '{IEEE} Transactions on Automation Science and Engineering',
  'pages': '1--1',
  'abstract': 'Real-world applications of nonlinear systems tracking control are always challenging due to the existence of uncertainties and disturbances. To design a robust optimal tracking controller for uncertain nonlinear systems with disturbances and actuator saturation, this paper investigates the prescribed performance robust optimal tr

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [140]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [141]:
def parse_author(a):
    """Parses author name in the format of `Last, First Middle` where
    the middle name is sometimes there and sometimes not (and could just be an initial)
    
    Returns: author name as `F. M. Last`
    """
    
    a = a.split(', ')
    last = a[0].strip()
    fm = a[1].split(' ')
    first = fm[0][0] + '.'
    
    if len(fm) > 1:
        middle = fm[1][0] + '.'
    else:
        middle = ''
#     print([first, middle, last])
    
    if not middle == '':
        return first + ' ' + middle + ' ' + last
    else:
        return first + ' ' + last

# for row, item in publications.iterrows():
#     authors = ', '.join([parse_author(a) for a in item['Author'].split('; ')])

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [142]:
p = publications.entries[-1]
p = bibtexparser.customization.add_plaintext_fields(p)
p

{'file': 'PDF:/Users/tanjunkai/Zotero/storage/G6B7QRJ7/Tan 等 - 2025 - Finite-time Safe Reinforcement Learning Control of Multi-player Nonzero-Sum Game for Quadcopter Syst.pdf:application/pdf;ScienceDirect Snapshot:/Users/tanjunkai/Zotero/storage/HNACTTAX/S002002552500249X.html:text/html',
 'keywords': 'adaptive dynamic programming, dynamic event-trigger, Finite-time optimal control, neural network, nonzero-sum game, reinforcement learning',
 'note': 'https://github.com/tanjunkai2001/{FT}-{SRL}-Quadcopter\n{EI}: 是',
 'date': '2025-03-24',
 'urldate': '2025-03-25',
 'author': 'Tan, Junkai and Xue, Shuangsi and Guan, Qingshu and Qu, Kai and Cao, Hui',
 'shortjournal': 'Information Sciences',
 'journaltitle': 'Information Sciences',
 'pages': '122117',
 'abstract': 'This paper investigates a finite-time safe reinforcement learning control algorithm for multi-player nonzero-sum games ({FT}-{SRL}-{NZS}). In addressing the finite-time safe optimal control issue, value functions incorporating 

In [143]:
import os


for item in publications.entries:
    
    item = bibtexparser.customization.add_plaintext_fields(item)
    year = item['plain_year'] if 'plain_year' in item else item['plain_date'].split('-')[0]
    key = item['plain_author'].split(',')[0].replace(' ', '').lower() + str(year) + item['plain_title'].split(' ')[0].lower()

    md_filename = key + ".md"
    html_filename = key
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item['plain_title'] + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
#     month = month_dict[item['plain_month']] if 'plain_month' in item else 6
#     day = item['plain_day'] if 'plain_day' in item else 15
#     date = datetime.date(int(year), int(month), int(day)).isoformat() + "00:00:00 + 0500"
    if not 'plain_date' in item:
        raise Exception(item)
    date = item['plain_date']
    if len(date) == 4:
        date += "-06-15 00:00:00 +0500"
    elif len(date) == 7:
        date += "-15 00:00:00 +0500"
    elif len(date) == 10:
        date += " 00:00:00 +0500"
    else:
        print(date)
        break
    
    
    md += "\ndate: " + date

    if 'plain_eventtitle' in item:
        venue = item['plain_eventtitle']
    elif 'plain_booktitle' in item:
        venue = item['plain_booktitle']
    elif 'plain_journaltitle' in item:
        venue = item['plain_journaltitle']
    elif 'plain_school' in item:
        venue = item['plain_institution']
    else:
        venue = False
    
    if 'plain_note' in item:
        note = item['plain_note']
        if 'submitted' in note.lower() or 'review' in note.lower() or 'accepted' in note.lower():
            venue += " (<b><i>" + note + "</i></b>)"
        
        
    if venue:
        md += "\nvenue: '" + html_escape(venue) + "'"
    
    if 'plain_url' in item:
        md += "\npaperurl: '" + item['plain_url'] + "'"
    
    if 'plain_doi' in item:
        md += "\ndoi: '" + item['plain_doi'] + "'"
        
    pubtypes = {"inproceedings": "conference",
                "article": "journal",
                "thesis": "academic"}
    md += "\npubtype: '" + pubtypes[item['ENTRYTYPE']] + "'"

    if item['ENTRYTYPE'] == 'article':
        # md += "\nimage: '" + venue + ".jpg'"
        md += "\nimage: '../images/" + venue + ".jpg'"

    if 'github' in note.lower():
        code = note.split('github.com/')[-1].split()[0]
        md += "\ncode: 'https://github.com/" + code + "'"

    citation = f"{item['plain_author']} ({year}). {item['plain_title']}. {venue}."
    md += "\ncitation: '" + html_escape(citation) + "'"
    authors = ', '.join([parse_author(a) for a in item['plain_author'].split(' and ')])
    md += "\nauthors: '" + authors + "'"
    
    md += "\nexcerpt_separator: \"\""
    
    md += "\n---"
    
    ## Markdown description for individual page
        
#     if len(str(item.excerpt)) > 5:
#         md += "\n" + html_escape(item.excerpt) + "\n"
    
    if 'plain_abstract' in item:
        md += "\n" + html_escape(item['plain_abstract']) + "\n"
    
    if 'plain_url' in item:
        md += "\n[Download paper here](" + item['plain_url'] + ")"
    
    if 'plain_doi' in item:
        md += "\n[DOI](" + item['plain_doi'] + ")"
        
#     md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [144]:
!ls ../_publications/

IEEE Transactions on Automation Science and Engineering.jpg
Information Sciences.jpg
International Journal of Robust and Nonlinear Control.jpg
Journal of Automation and Intelligence.jpg
Neurocomputing.jpg
Nonlinear Dynamics.jpg
tan2023nash.md
tan2023safe.md
tan2024safe.md
tan2025data-driven.md
tan2025finite-time.md
tan2025human–ai.md
tan2025human–machine.md
tan2025prescribed.md
tan2025stackelberg.md
tan2025unmanned.md


In [145]:
!cat ../_publications/behbahani2014aircraft.md

cat: ../_publications/behbahani2014aircraft.md: No such file or directory
